In [ ]:
from pylab import *
%matplotlib inline

The following function will compute the probability density as a function of both position and time given some initial state, $\psi(x, 0)$, and some potential energy function, $V(x)$.

In [ ]:
def evolve(realPsi0, imagPsi0, V_x, dx, dt, Nx, Nt):
    ''' Computes the time evolution of quantum wave packet
        Input:
            realPsi0: 1D array of real(psi) vs x at time t=0
            imagPsi0: 1D array of imag(psi) vs x at time t=0
            V_x: 1D array of potential energy V(x)
            dx: Discretization of space, delta x, in Angstroms
            dt: Value of the time step (in units of femtoseconds)
            Nt: Number of time steps to be simulated
        Output:
            2D complext array: psi versus t and x ([t] then [x])
    '''
    hbar_alpha = 0.1157 # hbar in units of m_electron*nm^2 / fs for use in alpha
    hbar_beta  = 0.6582 # hbar in units of eV * fs for use in beta
    
    alpha = hbar_alpha*dt / (2*dx**2)
    beta  = V_x * dt / hbar_beta
    
    # Create 2D arrays [time][space]
    psi = zeros( (Nt, Nx), dtype=np.complex128 ) # COMPLEX
    R   = zeros( (Nt, Nx) )
    I   = zeros( (Nt, Nx) )
    
    # Initial state:
    R[0] = realPsi0
    I[0] = imagPsi0
    
    # Initial half-step update for imaginary part
    for i in range(1, Nx-1):
        d2R = R[0][i-1] + R[0][i+1] - 2*R[0][i]
        deltaI = alpha*d2R - beta[i]*R[0][i]
        I[0][i] = I[0][i] + 0.5*deltaI # HALF of a time step (Euler)
    
    for n in range(1, Nt): # Loop over time,  n
        
        for i in range(1, Nx-1):  # Loop over space, i, to update the REAL PART
            d2I = I[n-1][i-1] + I[n-1][i+1] - 2*I[n-1][i]
            deltaR = -alpha*d2I + beta[i]*I[n-1][i]
            R[n][i] = R[n-1][i] + deltaR
                
        for i in range(1, Nx-1):  # Loop over space, i, to update the IMAGINARY
            d2R = R[n][i-1] + R[n][i+1] - 2*R[n][i]
            deltaI =  alpha*d2R - beta[i]*R[n][i]
            I[n][i] = I[n-1][i] + deltaI
        
        I_avg = 0.5*(I[n-1] + I[n]) # Use I from immediately before & after
        psi[n] = R[n] + 1j*I_avg    # Finishing one time step
    return psi

The following cell can be used to compile a function to make it faster, such that the compiled Python code will have execution times more like traditional compiled programming languages.

In [ ]:
from numba import complex128, int64, double, jit # "jit": "Just In Time compiler"
evolve_numba = jit(complex128[:,:]\
                   (double[:],double[:],double[:],double,double,int64,int64),\
                   nopython=True)(evolve)

In [ ]:
from time import sleep # To set the animation frame rate 
from IPython.display import clear_output # To redraw

def animate_tx(tArray, xArray, yArray, numFrames=20):
    '''Creates and displays an animation of z(x,t)
    Input:
        tArray: 1D array of time values
        xArray: 1D array of position values
        yArray: 2D array of y[time][position]
        (optional): # of frames to display, evenly
                    spaced from tArray[0] to tArray[-1]
    '''
    # Min & Max values for vertical axis of plot
    yMin = np.min(yArray)
    yMax = np.max(yArray)
    
    tNum = size(tArray) # Number of TOTAL time values...
    for i in range(0, tNum, tNum // (numFrames-1)):
        sleep(0.05) # Sets the maximum animation speed
        plot(xArray, yArray[i])
        ylim([yMin, yMax])
        xlabel('Position (nm)')
        ylabel('$\\rho(x)$ (1/nm)')
        titleString = "Time (fs): " + str(tArray[i])
        title(titleString)
        clear_output(wait=True)
        grid(True); show()

Starting with the initial wave packet,
$$\psi(x,0) = \frac{1}{\sqrt{a\sqrt{\pi}}} \exp\left(-\frac{(x-x_0)^2}{2a^2}\right) \exp(ik_0x) $$
where $a$, $x_0$, and $k_0$ are constant (user-defined) parameters.

In [ ]:
# PARAMETERS:
Nx = 512   # Number of x values (Need the ratio dt / (dx)^2 to be small)
Nt = 10001 # Number of t values (Need the ratio dt / (dx)^2 to be small)
L  = 10.0  # Right edge of "box" in nm
tMax = 12.0 # Maximum time in femtoseconds
x0 = 3.0   # Initial peak of gaussian in position basis
k0 = 10.0   # Initial peak of gaussian in monentum space
a  = 0.5   # Width of gaussian in position space

x = linspace(0, L, Nx)
V = zeros(Nx)

#V = 10*x

#V = zeros(Nx)
#V[230:260] = +3

dx = L    / (Nx - 1)
dt = tMax / (Nt - 1)

A = 1.0 / sqrt(a * sqrt(pi))
psi0 = A * exp(-(x-x0)**2 / (2*a**2)) * exp(1j*k0*x) # INTIAL QUANTUM STATE
realPsi0 = real(psi0)
imagPsi0 = imag(psi0)

# COMPUTE!
psi = evolve_numba(realPsi0, imagPsi0, V, dx, dt, Nx, Nt)
rho = abs(psi)**2
t = linspace(0, tMax, Nt) # Will be used for plotting

In [ ]:
# Checking normalization
from scipy.integrate import simps
simps(rho[-1], x)

In [ ]:
# Create animation
animate_tx(t, x, rho, numFrames=50)

In [ ]:
# Create 2D contour plot
contourf(x, t, rho, 100)
title('Probability Density (1/nm)')
xlabel('Position (nm)')
ylabel('Time (femtoseconds)')
colorbar()
show()

**FFT and Momentum Space:**

**Add detailed comments to describe what is happening in the following function "`transform(x, psi)`".**

In [ ]:
def transform(x, psi):
    Nt, Nx = shape(psi)
    dx = x[1]-x[0]
    k = linspace(-pi/dx, pi/dx, Nx)
    f = zeros( (Nt,Nx), dtype=np.complex128 )
    for i in range(Nt):
        f[i] = psi[i] * exp(-1j * x * k[0])
    
    psi_k = dx/sqrt(2*pi) * fft(f, axis=1)
    return k, psi_k

In [ ]:
k, psi_k = transform(x, psi)
rho_k = abs(psi_k)**2
print('Checking normalization:')
simps(rho_k[-1], k)

In [ ]:
def side_by_side_contour(x, rho, k, rho_k, t, kmin=-10, kmax=10):
    '''
    Displays side-by-side contour plots showing
    rho(x,t) and rho(k,t)'''
    
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
    
    ax1.contourf(x, t, rho, 100)
    ax1.set_xlabel('Position (nm)')
    ax1.set_ylabel('Time (femtoseconds)')
    ax1.set_title('$\\rho$(x,t) (1/nm)')
    ax1.grid(True)
    
    ax2.contourf(k, t, rho_k, 100)
    ax2.set_xlabel('k (1/nm)')
    ax2.set_title('$\\rho$(k,t) (nm)')
    ax2.grid(True)
    ax2.set_xlim([kmin, kmax])
    
    show()

def side_by_side_animation(x, rho, k, rho_k, t, \
                           numFrames = 20, kmin=-10, kmax=10):
    '''
    Displays side-by-side animations showing
    rho(x) and rho(k), each evolving with time.'''
    
    # Min & Max values for vertical axis of both plots
    min1 = np.min(rho)
    max1 = np.max(rho)
    min2 = np.min(rho_k)
    max2 = np.max(rho_k)
    
    tNum = size(t) # Number of TOTAL time values...
    for i in range(0, tNum, tNum // (numFrames-1)):
        sleep(0.05) # Sets the maximum animation speed
        
        f, (ax1, ax2) = plt.subplots(1, 2)
        ax1.plot(x, rho[i])
        ax1.set_ylim([min1, max1])
        ax1.set_xlabel('Position (nm)')
        ax1.set_ylabel('$\\rho(x)$ (1/nm)')
        ax1.grid(True)
        titleString = "Time (fs): " + str(t[i])
        ax1.set_title(titleString)
        
        ax2.plot(k, rho_k[i])
        ax2.set_xlim([kmin, kmax])
        ax2.set_ylim([min2, max2])
        ax2.set_xlabel('k (1/nm)')
        ax2.set_ylabel('$\\rho(k)$ (nm)')
        ax2.grid(True)
        
        tight_layout()
        clear_output(wait=True)
        show()

In [ ]:
side_by_side_animation(x, rho, k, rho_k, t, \
                       numFrames = 20, kmin=-3.5*k0, kmax=3.5*k0)

In [ ]:
side_by_side_contour(x, rho, k, rho_k, t, kmin=-3.5*k0, kmax=3.5*k0)